In [81]:
import os
import pandas as pd
import numpy as np
import pickle

# Setup

In [4]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://shai-nlp-course.netlify.app/clean-tweets.tsv"

In [5]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,أؤمن بأن الانسان ينطفئ جماله ابتعاد يحب بريق ا...,اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياة فنحن نعيش أرض أعدت للبلاء و...,خلو ضغط حية فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كآنت نقيه,نصم تسر حيت يرم لنس تعد كآن نقه


In [6]:
tweets = raw["clean_stemmed"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [7]:
sample = tweets.iloc[0]
sample

'اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذبل طفئ تحل ربع خرف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [84]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str]):
        self.vocab = self._create_vocab(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [10]:
sample_tokens = sample.split(" ")
sample_tokens

['اؤم',
 'بأن',
 'انس',
 'طفئ',
 'جمل',
 'بعد',
 'يحب',
 'برق',
 'عين',
 'خفي',
 'صبح',
 'ذبل',
 'طفئ',
 'تحل',
 'ربع',
 'خرف']

In [9]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

7754 61656


In [14]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'غطت': 0,
 'ونص': 1,
 'صوب': 2,
 'محن': 3,
 'لمب': 4,
 'عاجل؛': 5,
 'أولاد': 6,
 'نشن': 7,
 'كسن': 8,
 'شنط': 9,
 'ففي': 10,
 'يرو': 11,
 'كدر': 12,
 'ميت': 13,
 'شمس': 14,
 'طرح': 15,
 'ﻭأﻧﺖ': 16,
 'وغو': 17,
 'سعة': 18,
 'تلغ': 19,
 'ارس': 20,
 'تكر': 21,
 'ربغ': 22,
 'ضحت': 23,
 'شلت': 24,
 'عنصر': 25,
 'كدة': 26,
 'وقن': 27,
 'ضمن': 28,
 'شارع…': 29,
 'دائمـ': 30,
 'رضي': 31,
 'ثبث': 32,
 'هزة': 33,
 'عميد💛': 34,
 'حر': 35,
 'مرئ': 36,
 'رب…': 37,
 'انتخابية': 38,
 'قلبي❤كتير': 39,
 'بمه': 40,
 'عندج': 41,
 'ماسكيرانو': 42,
 'مني': 43,
 'نكن': 44,
 'دبر': 45,
 'تحم': 46,
 'حرك': 47,
 'طبه': 48,
 'يعن': 49,
 'هالمحب': 50,
 'سند': 51,
 'دكت': 52,
 'يدي': 53,
 'عكر': 54,
 'واصل': 55,
 'حوج': 56,
 'خفض': 57,
 'لفر': 58,
 'ؤون': 59,
 'تتح': 60,
 'شنيشل': 61,
 'كررش': 62,
 'ترتب': 63,
 'لسء': 64,
 'سن': 65,
 'سونج': 66,
 'وقراهافقط': 67,
 'لعم': 68,
 'حصة': 69,
 'طين': 70,
 'عدك': 71,
 'ممرات…': 72,
 'عمدو': 73,
 'عودته…': 74,
 'كدو': 75,
 'رد؛': 76,
 'ز…': 77,
 'ه🌿🖤': 78,
 'امع': 79,
 

In [93]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str]):
        super().__init__(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        all_word_tokens = set([token for sample in corpus for token in sample.split(" ")])
        vocab = {token: index for index, token in enumerate(word_level_tokens, start=1)} 
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [94]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets)

## Character Level Tokenization

In [16]:
sample_tokens = list(sample)
sample_tokens

['ر',
 'ق',
 'ب',
 ' ',
 'ن',
 'ق',
 'ل',
 ' ',
 'ب',
 'ش',
 'ر',
 ' ',
 'ن',
 'ش',
 'ر',
 ' ',
 'خ',
 'ب',
 'ر',
 ' ',
 'ر',
 'ي',
 'ض',
 ' ',
 'و',
 'ص',
 'ل',
 ' ',
 'ف',
 'ر',
 'ق',
 ' ',
 'ه',
 'و',
 'م',
 'ن',
 'ت',
 'م',
 ' ',
 'ب',
 'ط',
 'ل',
 ' ',
 'ع',
 'ر',
 'ب',
 ' ',
 'ن',
 'ط',
 'ق',
 ' ',
 'ب',
 'ر',
 'ج',
 ' ',
 'ح',
 'م',
 'د',
 ' ',
 '📍']

In [17]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

346


In [18]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'ﻟ': 0,
 'ج': 1,
 'ﺷ': 2,
 'س': 3,
 'ﺍ': 4,
 '🚦': 5,
 '😆': 6,
 '🏼': 7,
 '😟': 8,
 '⁰': 9,
 'ﻌ': 10,
 'ﺎ': 11,
 'ﺫ': 12,
 '😀': 13,
 '⚘': 14,
 '😂': 15,
 '💪': 16,
 'ี': 17,
 'م': 18,
 'ﻠ': 19,
 '💵': 20,
 '❓': 21,
 '״': 22,
 '🙂': 23,
 '❤': 24,
 'ا': 25,
 '😘': 26,
 '↷': 27,
 '✈': 28,
 '⠀': 29,
 '💡': 30,
 'ﻏ': 31,
 '☕': 32,
 '😑': 33,
 '✔': 34,
 '💀': 35,
 '♓': 36,
 '🇶': 37,
 '🍁': 38,
 '۞': 39,
 'ﺃ': 40,
 '😒': 41,
 '🍀': 42,
 '🙃': 43,
 '🤔': 44,
 'ﺸ': 45,
 '💴': 46,
 '🌿': 47,
 '￼': 48,
 '🏴': 49,
 '💶': 50,
 'ร': 51,
 '👆': 52,
 'ﺣ': 53,
 'ة': 54,
 'ﻔ': 55,
 '▪': 56,
 'ڪ': 57,
 '😐': 58,
 '🙆': 59,
 '🌚': 60,
 'ص': 61,
 '”': 62,
 '♥': 63,
 '🇷': 64,
 'ﺮ': 65,
 'ﻣ': 66,
 'ذ': 67,
 '😶': 68,
 '☺': 69,
 'ﻢ': 70,
 'ﻨ': 71,
 '⸀': 72,
 '💔': 73,
 '·': 74,
 '🏻': 75,
 '🕋': 76,
 'ﺬ': 77,
 '💞': 78,
 '🤷': 79,
 '😞': 80,
 '💖': 81,
 'ﷻ': 82,
 '💜': 83,
 'ﻃ': 84,
 'ﺟ': 85,
 '🎁': 86,
 'ﺨ': 87,
 '\u2069': 88,
 '»': 89,
 '🍃': 90,
 '💗': 91,
 '⭕': 92,
 '🏀': 93,
 '👮': 94,
 'ﻜ': 95,
 '😥': 96,
 'า': 97,
 '⚽': 98,
 '🌷': 99,
 'ٱ':

In [71]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document 
class CharacterLevelTokenizer(Tokenizer):
    ...

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [74]:
tokenized_tweets = word_level_tokenizer.tokenize(tweets)

In [88]:
max([len(t) for t in tokenized_tweets])

25

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [83]:
word_level_tokenizer_path = os.path.join(data_dir, "word_tokenizer.pkl")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)


In [85]:
# TODO: Pickle character level tokenizer